## Hagamos calculos en tiempo real

Primero, importemos las librerias que nos interesa.

In [1]:
import pandas as pd
import numpy as np

Luego llega el momento de empezar a explorar el conjunto de datos que queremos trabajar:

* Primero debemos cargar el documento fuente en memoria.
* Segundo, exploremos una pequeña muestra del conjunto de datos y miremos como se ven.

In [2]:
df = pd.read_excel("sources/libro_auxiliar.xlsx")
df.head()

,MOBIUS S.A.S.,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8
0,Libro Auxiliar entre el 01/01/2016 y el 31/12/...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NomCuenta,Tercero,Fecha,Notas,ChequeNumero,Doc,Débitos,Créditos,Saldo
2,11100501 Cuenta Corriente Davivienda cta. No. ...,BANCO DAVIVIENDA,2016-02-02 00:00:00,Apertura de cuenta corriente,NaN,(MS) NC 4,30000000,0,30000000
3,NaN,BANCO DAVIVIENDA,2016-02-02 00:00:00,Pago compra chequera,NaN,(MS) NC 4,0,392241,29607759
4,NaN,BANCO DAVIVIENDA,2016-02-02 00:00:00,IVA Chequera,NaN,(MS) NC 4,0,62759,29545000


Se aprecia que realmente el conjunto de datos tiene problemas, el principal parece estar relacionado con el hecho del formato de la fuente, corrijamoslo:

* Primero, cargamos nuevamente el documento en un DataFrame de Pandas, pero en esta ocasion saltamos las filas de datos vacias.
* Segundo, demos una mirada mas profunda a todo el conjunto.

In [3]:
df = pd.read_excel("sources/libro_auxiliar.xlsx", skiprows=2)
df

,NomCuenta,Tercero,Fecha,Notas,ChequeNumero,Doc,Débitos,Créditos,Saldo
0,11100501 Cuenta Corriente Davivienda cta. No. ...,BANCO DAVIVIENDA,2016-02-02,Apertura de cuenta corriente,NaN,(MS) NC 4,3.000000e+07,0.000000e+00,3.000000e+07
1,NaN,BANCO DAVIVIENDA,2016-02-02,Pago compra chequera,NaN,(MS) NC 4,0.000000e+00,3.922410e+05,2.960776e+07
2,NaN,BANCO DAVIVIENDA,2016-02-02,IVA Chequera,NaN,(MS) NC 4,0.000000e+00,6.275900e+04,2.954500e+07
3,NaN,BANCO DAVIVIENDA,2016-02-02,4x1000,NaN,(MS) NC 4,0.000000e+00,2.462000e+03,2.954254e+07
4,NaN,WORLD OFFICE COLOMBIA SAS,2016-02-25,Pago licencia software contable World Office,8249.0,(MS) ce 5,0.000000e+00,3.262500e+05,2.921629e+07
5,NaN,Aguirre Arias Julian Osiris,2016-02-25,Pago asesoria financiera enero y febrero 2016,8251.0,(MS) ce 7,0.000000e+00,1.000000e+06,2.821629e+07
6,NaN,Camara de Comercio de Cali,2016-02-25,Pago Renovacion de Camara de Comercio 2016,NaN,(MS) ce 8,0.000000e+00,2.892000e+05,2.792709e+07
7,NaN,Mosos Martinez William Andres,2016-02-26,Pago con cheque honorarios William Mosos,8250.0,(MS) ce 6,0.000000e+00,1.350000e+06,2.657709e+07
8,NaN,Leon Villamizar Ernesto,2016-03-03,Pago excedente cuenta de cobro,NaN,(MS) ce 15,0.000000e+00,1.958000e+06,2.461909e+07
9,NaN,BANCO DAVIVIENDA,2016-03-03,Pago seguridad social febrero 2016,8254.0,(MS) ce 16,0.000000e+00,3.600000e+06,2.101909e+07


Evidentemente podemos limpiar (*scrubingun* es el termino preferido) un poco mas los datos.

* Primero, identificamos filas y columnas de datos vacios, que introducen ruido o que son innecesarias y los descartamos.
* Segundo, modificamos la estructura de datos para que tenga mejor lectura y se manipulen mejor los datos.
* Tercero, volvamos a mirar.

In [4]:
# Comando para descartar filas con datos de tipo Null
df = df[df.Fecha.notnull()]

# Comando para descartar filas que contienen un "string" especifico.
df[df.NomCuenta.str.contains("Total") == False]

# Comando para descartar columnas dentro del conjunto de datos.
df.drop(['ChequeNumero', 'Doc'], axis=1, inplace=True)

# Cambiamos los nombres de algunas de las columnas.
df.rename(inplace=True, columns = {
                                    'NomCuenta':'cuenta_detallada', 
                                    'Tercero':'tercero',
                                    'Notas':'detalle', 'Débitos':'debitos', 
                                    'Créditos':'creditos', 'Saldo':'saldo'
                                })

# Creamos un indice pivotable a partir de una columna.
df['mes'] = pd.DatetimeIndex(df['Fecha']).month

# Rellenamos
df.cuenta_detallada.fillna(method='ffill', inplace=True)
df['clase'] = df.cuenta_detallada.str[:1]
df['grupo'] = df.cuenta_detallada.str[:2]
df['cuenta'] = df.cuenta_detallada.str[:4]
df['subcuenta'] = df.cuenta_detallada.str[:6]
df['cuenta_especifica'] = df.cuenta_detallada.str[:8]
df['nombre_cuenta'] = df.cuenta_detallada.str[7:]
df['consolidado'] = df.creditos - df.debitos
#df = df[df.mes != 12] # This line drops december of previous year, but careful because it can drop actual december 
#df.index = df.mes
df = df[['clase', 'grupo', 'cuenta', 'subcuenta', 'cuenta_especifica', 'nombre_cuenta', 'cuenta_detallada',
         'tercero', 'Fecha', 'detalle', 'debitos', 'creditos', 'saldo', 'consolidado', 'mes']]
df.head()

/home/maestro/Workspace/presentations/env/lib/python3.5/site-packages/ipykernel/__main__.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/maestro/Workspace/presentations/env/lib/python3.5/site-packages/pandas/core/frame.py:2754: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  **kwargs)
/home/maestro/Workspace/presentations/env/lib/python3.5/site-packages/ipykernel/__main__.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/

,clase,grupo,cuenta,subcuenta,cuenta_especifica,nombre_cuenta,cuenta_detallada,tercero,Fecha,detalle,debitos,creditos,saldo,consolidado,mes
0,1,11,1110,111005,11100501,1 Cuenta Corriente Davivienda cta. No. 0166699...,11100501 Cuenta Corriente Davivienda cta. No. ...,BANCO DAVIVIENDA,2016-02-02,Apertura de cuenta corriente,30000000.0,0.0,30000000.0,-30000000.0,2
1,1,11,1110,111005,11100501,1 Cuenta Corriente Davivienda cta. No. 0166699...,11100501 Cuenta Corriente Davivienda cta. No. ...,BANCO DAVIVIENDA,2016-02-02,Pago compra chequera,0.0,392241.0,29607759.0,392241.0,2
2,1,11,1110,111005,11100501,1 Cuenta Corriente Davivienda cta. No. 0166699...,11100501 Cuenta Corriente Davivienda cta. No. ...,BANCO DAVIVIENDA,2016-02-02,IVA Chequera,0.0,62759.0,29545000.0,62759.0,2
3,1,11,1110,111005,11100501,1 Cuenta Corriente Davivienda cta. No. 0166699...,11100501 Cuenta Corriente Davivienda cta. No. ...,BANCO DAVIVIENDA,2016-02-02,4x1000,0.0,2462.0,29542538.0,2462.0,2
4,1,11,1110,111005,11100501,1 Cuenta Corriente Davivienda cta. No. 0166699...,11100501 Cuenta Corriente Davivienda cta. No. ...,WORLD OFFICE COLOMBIA SAS,2016-02-25,Pago licencia software contable World Office,0.0,326250.0,29216288.0,326250.0,2


Ahora hagamos algo aun mas chevere, reemplacemos Excel.

Creemos una *pivot table* de Pandas (es igual a una tabla dinamica... en Python...

**BUUUUUUMMMMM** (La cabeza explota)

In [5]:
# Definimos la estructura del queryset a manejar.
query = 'cuenta == ['
cuentas = (4105, 4110, 4115, 4120, 4125, 4130, 4135, 4140, 4145, 4150, 4155, 4160, 4165,
           4170, 4175, 4205, 4210, 4215, 4220, 4225, 4230, 4235, 4240, 4245, 4248, 4250,
           4255, 4260, 4265, 4275, 4295, 4705, 5105, 5110, 5115, 5120, 5125, 5130, 5135,
           5140, 5145, 5150, 5155, 5160, 5165, 5195, 5199, 5205, 5210, 5215, 5220, 5225,
           5230, 5235, 5240, 5245, 5250, 5255, 5260, 5265, 5270, 5295, 5299, 5305, 5310,
           5315, 5395, 5405, 5905, 6105, 6110, 6115, 6120, 6125, 6130, 6135, 6140, 6145,
           6150, 6155, 6160, 6165, 6170, 6205, 6210, 6215, 6220, 6225, 7105, 7110, 7115,
           7120, 7125, 7130, 7135, 7140, 7145, 7150, 7155, 7160, 7165, 7170, 7175, 7180,
           7185, 7190, 7195, 7200, 7205, 7210, 7215, 7220, 7225, 7230, 7235, 7240, 7245,
           7250, 7255, 7260, 7265, 7270, 7275, 7280, 7285, 7290, 7295, 7300, 7305, 7310,
           7315, 7320, 7325, 7330, 7335, 7340, 7345, 7350, 7355, 7360, 7365, 7370, 7375,
           7380, 7385, 7390, 7395)

indice = []
for i in cuentas:
    indice.append(str(i))

for i in cuentas:
    i = '"' + str(i) + '", '
    query = query + i

query = query + ']'
tb = pd.pivot_table(df, index=['cuenta', 'nombre_cuenta'], values=['consolidado'], 
                    columns=['mes'], aggfunc=np.sum, fill_value=0).query(query)

# Y veamos como queda esta cosa.
tb

consolidado  \
mes                                                                1    
cuenta nombre_cuenta                                                    
4135   VENTA DE OTROS PRODUCTOS                                  0.00   
4155   ACTIVIDADES EMPRESARIALES DE CONSULTORIA                  0.00   
4210   INTERESES                                               809.63   
4295   AJUSTE AL PESO                                            0.00   
       APROVECHAMIENTOS                                          0.00   
5105   APORTES A ADMINISTRADORAS DE RIESGOS PROFESIONA...   -12246.00   
       APORTES A FONDOS DE PENSIONES Y/0 CESANTIAS         -281520.00   
       APORTES CAJAS DE COMPENSACION FAMILIAR               -93840.00   
       CAPACITACION AL PERSONAL                                  0.00   
       CESANTIAS                                           -195492.00   
       INTERESES SOBRE CESANTIAS                            -23460.00   
       PRIMA DE SERVICIOS                                  -195492.00   
       SUELDOS                                            -2346000.00   
       VACACIONES                                           -97758.00   
5110   ASESORIA FINANCIERA                                -1000000.00   
5115   OTROS                                                -12930.00   
5130   RESPONSABILIDAD CIVIL Y EXTRACONTRACTUAL            -346779.00   
5135   CORREO, PORTES Y TELEGRAMAS                               0.00   
       OTROS                                                     0.00   
       PROCESAMIENTO ELECTRONICO DE DATOS                        0.00   
5140   REGISTRO MERCANTIL                                        0.00   
5155   PASAJES AEREOS                                      -937800.00   
5195   1 Medios de transporte                              -938400.00   
       1 PAPELERIA                                         -148448.00   
       TAXIS Y BUSES                                       -744022.00   
       UTILES, PAPELERIA Y FOTOCOPIAS                            0.00   
5205   APORTES A ADMINISTRADORAS DE RIESGOS PROFESIONA...  -278172.00   
       APORTES AL I.S.S                                          0.00   
       APORTES CAJAS DE COMPENSACION FAMILIAR              -456583.00   
       APORTES I.C.B.F.                                          0.00   
...                                                               ...   
       INTERESES SOBRE CESANTIAS                           -114146.00   
       PENSIONES                                          -1369748.00   
       PRIMA DE SERVICIOS                                  -951175.00   
       SENA                                                      0.00   
       SUELDOS                                            -5212000.00   
       VACACIONES                                          -475588.00   
5210   OTROS                                                     0.00   
5215   INDUSTRIA Y COMERCIO                                      0.00   
5235   ASISTENCIA TECNICA                                        0.00   
       OTROS                                                     0.00   
       TRANSPORTE, FLETES Y ACARREOS                             0.00   
5255   ALOJAMIENTO Y MANUTENCION                                 0.00   
       OTROS                                                     0.00   
       PASAJES AEREOS                                            0.00   
       PASAJES TERRESTRES                                        0.00   
5295   1 Medios de transporte                             -2084800.00   
       CASINO Y RESTAURANTE                                      0.00   
       TAXIS Y BUSES                                             0.00   
5305   1 COMISIONES BANCARIAS                                    0.00   
       1 Graven al movimiento financiero                    -60400.00   
       DIFERENCIA EN CAMBIO                                      0.00   
       GASTOS BANCARIOS                    